# 从零实现感知器

## 📚 代码示例关联

**本示例位置**：这是神经网络基础的第一个代码示例

**基于**：无（这是起点）

**扩展为**：
- [从零实现神经元](../01_从零实现神经元.ipynb) - 感知器是神经元的特例，神经元添加了激活函数
- [从零实现前馈神经网络](../../02_前馈神经网络/代码示例/01_从零实现前馈神经网络.ipynb) - 多层感知器（MLP）由多个感知器组成

**相关示例**：
- [理解感知器基本操作](../../练习题/基础练习/练习1_理解感知器基本操作.ipynb) - 基础练习
- [感知器分类器](../../实战案例/简单项目1_感知器分类器/main.ipynb) - 实战应用

---

## 学习目标
- 理解感知器的基本结构和工作原理
- 掌握感知器学习算法
- 能够从零实现感知器
- 可视化感知器的决策边界和学习过程

## 课程概述
本notebook将带你从零开始实现感知器算法，包括：
1. 感知器的基本结构
2. 感知器学习算法
3. 可视化决策边界
4. 分析学习过程


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 设置随机种子，确保结果可复现
np.random.seed(42)

print("环境准备完成！")


## 1. 感知器类实现

我们将从零实现一个完整的感知器类，包含训练和预测功能。


In [ ]:
class Perceptron:
    """
    感知器类
    从零实现感知器算法，适合小白学习
    """
    
    def __init__(self, learning_rate=0.01, max_iter=1000):
        """
        初始化感知器
        参数:
            learning_rate: 学习率，控制权重更新的步长（默认0.01）
            max_iter: 最大迭代次数，防止无限循环（默认1000）
        """
        self.learning_rate = learning_rate  # 学习率
        self.max_iter = max_iter  # 最大迭代次数
        self.weights = None  # 权重向量（待训练）
        self.bias = None  # 偏置（待训练）
        self.errors = []  # 记录每次迭代的错误数（用于可视化学习过程）
    
    def fit(self, X, y):
        """
        训练感知器
        参数:
            X: 特征矩阵，形状为(n_samples, n_features)
               n_samples是样本数，n_features是特征数
            y: 标签向量，形状为(n_samples,)，值为0或1
        """
        # 获取数据维度
        n_samples, n_features = X.shape
        
        # 初始化权重和偏置
        # 权重：随机初始化，范围在[-0.5, 0.5]
        # 这样可以让初始权重不会太大，有利于训练
        self.weights = np.random.uniform(-0.5, 0.5, n_features)
        # 偏置：初始化为0
        self.bias = 0.0
        
        print("=" * 60)
        print("开始训练感知器...")
        print("=" * 60)
        print(f"数据维度: {n_samples} 个样本, {n_features} 个特征")
        print(f"初始权重: {self.weights}")
        print(f"初始偏置: {self.bias}")
        print()
        
        # 训练循环
        for iteration in range(self.max_iter):
            errors = 0  # 记录本次迭代的错误数
            
            # 遍历每个训练样本
            for i in range(n_samples):
                # 步骤1：计算加权和（净输入）
                # z = w1*x1 + w2*x2 + ... + wn*xn + b
                # 使用点积计算：w^T * x + b
                z = np.dot(X[i], self.weights) + self.bias
                
                # 步骤2：通过阶跃函数得到预测值
                # 如果z >= 0，输出1；否则输出0
                # 这就是感知器的激活函数（阶跃函数）
                prediction = 1 if z >= 0 else 0
                
                # 步骤3：如果预测错误，更新权重和偏置
                if prediction != y[i]:
                    errors += 1  # 记录错误
                    
                    # 更新权重
                    # 如果y[i]=1但prediction=0，需要增加权重（使z变大）
                    # 如果y[i]=0但prediction=1，需要减少权重（使z变小）
                    # 更新公式：w = w + learning_rate * (y - prediction) * x
                    self.weights += self.learning_rate * (y[i] - prediction) * X[i]
                    
                    # 更新偏置
                    # 偏置的更新与权重类似
                    self.bias += self.learning_rate * (y[i] - prediction)
            
            # 记录本次迭代的错误数（用于可视化）
            self.errors.append(errors)
            
            # 每10次迭代打印一次进度
            if (iteration + 1) % 10 == 0:
                print(f"迭代 {iteration + 1}: 错误数 = {errors}")
            
            # 如果所有样本都分类正确，提前结束（收敛）
            if errors == 0:
                print(f"\n✓ 训练完成！在第 {iteration + 1} 次迭代时收敛")
                print(f"最终权重: {self.weights}")
                print(f"最终偏置: {self.bias}")
                break
        
        # 如果达到最大迭代次数仍有错误，给出警告
        if errors > 0:
            print(f"\n⚠ 警告：达到最大迭代次数 {self.max_iter}，仍有 {errors} 个错误")
            print("可能原因：数据不是线性可分的")
    
    def predict(self, X):
        """
        预测
        参数:
            X: 特征矩阵，形状为(n_samples, n_features)
        返回:
            predictions: 预测结果，形状为(n_samples,)，值为0或1
        """
        # 计算加权和（对所有样本）
        # X是(n_samples, n_features)，weights是(n_features,)
        # 使用矩阵乘法：X @ weights 得到(n_samples,)
        z = np.dot(X, self.weights) + self.bias
        
        # 通过阶跃函数得到预测值
        # z >= 0 返回True（转换为1），否则返回False（转换为0）
        predictions = (z >= 0).astype(int)
        
        return predictions
    
    def score(self, X, y):
        """
        计算准确率
        参数:
            X: 特征矩阵
            y: 真实标签
        返回:
            accuracy: 准确率（0到1之间）
        """
        # 进行预测
        predictions = self.predict(X)
        
        # 计算准确率：预测正确的样本数 / 总样本数
        accuracy = np.mean(predictions == y)
        
        return accuracy

print("感知器类定义完成！")


## 2. 生成测试数据

我们生成一个简单的线性可分的二分类数据集来测试感知器。


In [ ]:
# 生成简单的线性可分数据
# 正类样本（标签=1）：分布在右上角
np.random.seed(42)
X1 = np.random.randn(50, 2) + [2, 2]  # 均值在(2, 2)，标准差为1
y1 = np.ones(50)  # 标签为1

# 负类样本（标签=0）：分布在左下角
X0 = np.random.randn(50, 2) + [-2, -2]  # 均值在(-2, -2)，标准差为1
y0 = np.zeros(50)  # 标签为0

# 合并数据
X = np.vstack([X1, X0])  # 垂直堆叠，得到(100, 2)
y = np.hstack([y1, y0])  # 水平堆叠，得到(100,)

print("=" * 60)
print("数据信息")
print("=" * 60)
print(f"总样本数: {len(X)}")
print(f"特征数: {X.shape[1]}")
print(f"正类样本数: {np.sum(y == 1)}")
print(f"负类样本数: {np.sum(y == 0)}")
print(f"数据范围: X1在[{X[:, 0].min():.2f}, {X[:, 0].max():.2f}], "
      f"X2在[{X[:, 1].min():.2f}, {X[:, 1].max():.2f}]")

# 可视化数据分布
plt.figure(figsize=(8, 6))
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', marker='o', 
           label='正类 (y=1)', alpha=0.6, s=50)
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', marker='s', 
           label='负类 (y=0)', alpha=0.6, s=50)
plt.xlabel('特征1 (x1)', fontsize=12)
plt.ylabel('特征2 (x2)', fontsize=12)
plt.title('训练数据分布', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 3. 训练感知器

现在使用生成的数据训练感知器。


In [ ]:
# 创建感知器实例
# learning_rate: 学习率，控制权重更新的步长
# max_iter: 最大迭代次数
perceptron = Perceptron(learning_rate=0.1, max_iter=100)

# 训练感知器
perceptron.fit(X, y)

# 评估训练集上的准确率
train_accuracy = perceptron.score(X, y)
print(f"\n训练集准确率: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")


## 4. 可视化学习过程

观察感知器在学习过程中错误数的变化。


In [ ]:
# 可视化训练过程
plt.figure(figsize=(10, 5))

# 左图：错误数随迭代次数的变化
plt.subplot(1, 2, 1)
plt.plot(perceptron.errors, linewidth=2, color='blue')
plt.xlabel('迭代次数', fontsize=12)
plt.ylabel('错误数', fontsize=12)
plt.title('训练过程：错误数随迭代次数的变化', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--', linewidth=1, label='收敛（错误数=0）')
plt.legend()

# 右图：累积错误数
plt.subplot(1, 2, 2)
cumulative_errors = np.cumsum(perceptron.errors)
plt.plot(cumulative_errors, linewidth=2, color='green')
plt.xlabel('迭代次数', fontsize=12)
plt.ylabel('累积错误数', fontsize=12)
plt.title('累积错误数', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"总迭代次数: {len(perceptron.errors)}")
print(f"总错误数: {sum(perceptron.errors)}")


## 5. 可视化决策边界

绘制感知器学习到的决策边界，观察它如何将两类数据分开。


In [ ]:
# 可视化决策边界
plt.figure(figsize=(10, 8))

# 绘制数据点
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', marker='o', 
           label='正类 (y=1)', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', marker='s', 
           label='负类 (y=0)', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

# 绘制决策边界
# 决策边界方程：w1*x1 + w2*x2 + b = 0
# 即：x2 = -(w1*x1 + b) / w2
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x1_line = np.linspace(x1_min, x1_max, 100)
x2_line = -(perceptron.weights[0] * x1_line + perceptron.bias) / perceptron.weights[1]

plt.plot(x1_line, x2_line, 'g--', linewidth=3, label='决策边界', alpha=0.8)

# 绘制权重向量方向（垂直于决策边界）
# 权重向量指向正类区域
center_x, center_y = X.mean(axis=0)
arrow_length = 2
plt.arrow(center_x, center_y, 
         perceptron.weights[0] * arrow_length, 
         perceptron.weights[1] * arrow_length,
         head_width=0.3, head_length=0.2, fc='orange', ec='orange', 
         linewidth=2, label='权重向量方向')

plt.xlabel('特征1 (x1)', fontsize=12)
plt.ylabel('特征2 (x2)', fontsize=12)
plt.title('感知器分类结果和决策边界', fontsize=14)
plt.legend(fontsize=10, loc='best')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

# 打印决策边界方程
print("=" * 60)
print("决策边界方程")
print("=" * 60)
print(f"{perceptron.weights[0]:.4f} * x1 + {perceptron.weights[1]:.4f} * x2 + {perceptron.bias:.4f} = 0")
print(f"或简化为: x2 = -({perceptron.weights[0]:.4f} * x1 + {perceptron.bias:.4f}) / {perceptron.weights[1]:.4f}")


## 6. 测试新样本

使用训练好的感知器预测新样本的类别。


In [ ]:
# 生成一些新的测试样本
test_samples = np.array([
    [3, 3],    # 应该在正类区域
    [-3, -3],  # 应该在负类区域
    [0, 0],    # 在边界附近
    [1, 1],    # 在正类区域
    [-1, -1]   # 在负类区域
])

# 进行预测
predictions = perceptron.predict(test_samples)

print("=" * 60)
print("新样本预测结果")
print("=" * 60)
for i, (sample, pred) in enumerate(zip(test_samples, predictions)):
    print(f"样本 {i+1}: {sample} -> 预测类别: {pred} ({'正类' if pred == 1 else '负类'})")

# 可视化测试样本
plt.figure(figsize=(10, 8))
# 绘制训练数据
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', marker='o', 
           label='训练数据-正类', alpha=0.3, s=30)
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', marker='s', 
           label='训练数据-负类', alpha=0.3, s=30)

# 绘制测试样本
colors = ['green' if p == 1 else 'purple' for p in predictions]
plt.scatter(test_samples[:, 0], test_samples[:, 1], c=colors, 
           marker='*', s=200, edgecolors='black', linewidth=2,
           label='测试样本', zorder=5)

# 绘制决策边界
x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
x1_line = np.linspace(x1_min, x1_max, 100)
x2_line = -(perceptron.weights[0] * x1_line + perceptron.bias) / perceptron.weights[1]
plt.plot(x1_line, x2_line, 'g--', linewidth=2, label='决策边界', alpha=0.8)

plt.xlabel('特征1 (x1)', fontsize=12)
plt.ylabel('特征2 (x2)', fontsize=12)
plt.title('新样本预测结果', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 7. 总结与思考

### 关键知识点总结

1. **感知器的基本结构**：
   - 输入特征、权重、偏置、激活函数（阶跃函数）
   - 输出：0或1（二分类）

2. **感知器学习算法**：
   - 初始化权重和偏置
   - 逐个样本训练，预测错误时更新权重
   - 直到所有样本分类正确或达到最大迭代次数

3. **决策边界**：
   - 线性决策边界（直线或超平面）
   - 权重向量垂直于决策边界

4. **局限性**：
   - 只能解决线性可分的二分类问题
   - 无法解决XOR等非线性问题

### 思考问题

1. 为什么感知器只能解决线性可分问题？
2. 学习率对训练过程有什么影响？尝试修改学习率观察效果。
3. 如果数据不是线性可分的，感知器会怎样？
4. 如何解决感知器的局限性？（提示：多层神经网络）

### 下一步学习

- 学习更复杂的神经元模型（Sigmoid、ReLU等激活函数）
- 学习如何组合多个感知器（多层神经网络）
- 学习反向传播算法（如何训练多层网络）

---

**恭喜！你已经掌握了感知器的基本原理和实现！** 🎉
